##1. Install nltk, download the data

I've uploaded the data and the validation datasets to [transfer.sh](https://transfer.sh) and Unix curl it so you don't have to mount Drive. If you are checking this **after November 12th**, that is more than 2 weeks after the submission deadline, please reupload the identical Simpsons2022.zip and the development-examples.txt, change the urls in the bellow cell, and do Runtime => Disconnect and delete runtime before rerunning because Unix may hang for some reason.

In [ ]:
# %%capture
# %%capture suppresses output
!pip install nltk
!git clone https://gist.github.com/sebleier/554280 # for a stopword list that's better
# suited this task than NLTK.download('stopwords') stopwords.words('english') (I don't think 'won' semantically is an English stopword)
!curl https://transfer.sh/orAe3G/Simpsons-2022%281%29.zip -o /content/simpons.zip
!curl https://transfer.sh/dCrYC2/development-examples%282%29.txt -o /content/development-examples.txt
!mkdir -p /content/simpsons_data/
!unzip /content/simpons.zip -d /content/simpsons_data
!rm -r /content/simpsons_data/__MACOSX
!rm /content/simpons.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into '554280'...
remote: Enumerating objects: 3, done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (3/3), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  738k  100  738k    0     0   533k      0  0:00:01  0:00:01 --:--:--  532k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   154  100   154    0     0    331      0 --:--:-- --:--:-- --:--:--   331
Archive:  /content/simpons.zip
  inflating: /content/simpsons_data/3.1.txt  
  inflating: /content/simpsons_data/__MACOSX/._3.1.txt  
  inflating: /content/simpsons_data/3.2.txt  
  inflating: /content/simpsons_data/__MACOSX/._3.2.txt  
  inflating: /content/simpson

##2. Import libs, define constants, set contractions map and Wikipedia filter list

The contractions map is from https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/bonus%20content/nlp%20proven%20approach/contractions.py.

In [ ]:
%%capture
import os
import re
import time
from collections import Counter
import csv
from pprint import pprint
from tqdm.notebook import tqdm # for progress bars
from nltk import word_tokenize, download, PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
download('stopwords')
download('punkt')
download('omw-1.4')
download('wordnet')

# ENGLISH_STOPWORDS = set(stopwords.words('english')) # no!
ENGLISH_STOPWORDS = (open("/content/554280/NLTK's list of english stopwords").read().split('\n'))
INPUT_FOLDER_PATH = '/content/simpsons_data'

# from https://github.com/dipanjanS/practical-machine-learning-with-python/blob/master/bonus%20content/nlp%20proven%20approach/contractions.py
"""
@author: DIP
"""

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

# re.compile object to help to find document places with contractions
CONTRACTIONS_PATTERN = re.compile(
    "({})".format("|".join(CONTRACTION_MAP.keys())),
    flags=re.IGNORECASE | re.DOTALL,
)

WIKIPEDIA_MAP = [
("From Wikipedia, the free encyclopedia", " "),
("Wikipedia", " "),
("Jump to navigation", " "),
("Jump to search", ' '),
("Contents\n", " "),
("Plot\n", " "),
("Production\n", " "),
("Alternate opening\n", " "),
("Unproduced sequel\n", " "),
("Cultural references\n", " "),
("Reception\n", " "),
("Reruns\n", " "),
("Pull from circulation\n", " "),
("References\n", " "),
("External links\n", " "),
]

https://gist.github.com/sebleier/554280:

In [ ]:
print(ENGLISH_STOPWORDS)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

##3. Define Norvig spell checker, construct a .txt file with all episodes (for spell checking)
Spell checker based on https://norvig.com/spell-correct.html. For spell checking, all .txt files are just joined with a newline and saved as /content/big_simpsons.txt (run this cell first to access it).

In [ ]:
def norvig_correction(word): 
# from https://norvig.com/spell-correct.html
    "Most probable spelling correction for word."
    def words(text): return re.findall(r'\w+', text.lower())
    WORDS = Counter(words(open('/content/big_simpsons.txt').read()))
    def known(words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in WORDS)
    def P(word, N=sum(WORDS.values())): 
        "Probability of `word`."
        return WORDS[word] / N
    def candidates(word): 
        "Generate possible spelling corrections for word."
        def edits1(word):
            "All edits that are one edit away from `word`."
            letters    = 'abcdefghijklmnopqrstuvwxyz'
            splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
            deletes    = [L + R[1:]               for L, R in splits if R]
            transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
            replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
            inserts    = [L + c + R               for L, R in splits for c in letters]
            return set(deletes + transposes + replaces + inserts)
        def edits2(word): 
            "All edits that are two edits away from `word`."
            return (e2 for e1 in edits1(word) for e2 in edits1(e1))
        return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
    return max(candidates(word), key=P)

convert = (lambda text: int(text) if text.isdigit() else text.lower()) # So that episodes in the txt are saved alphanumerically by season and then episode
big_simpsons_string = 'This is the full collection raw .txt files in the corpus\n' # First line in the txt
for file_name in (
    sorted(
        os.listdir(INPUT_FOLDER_PATH),
        key=lambda key: [
            convert(c) for c in re.split("([0-9]+)", key)
        ],  # sorted first by season, then by episode
    )
):
    if file_name.endswith(".txt"):
        episode_string = open(os.path.join(INPUT_FOLDER_PATH, file_name)).read()
        big_simpsons_string += '\n' + episode_string #Append current episode to the large string
big_simpsons_txt = open("/content/big_simpsons.txt", "w")
big_simpsons_txt.write(big_simpsons_string)
big_simpsons_txt.close()

##4. InvertedIndex class

In [ ]:
class InvertedIndex:
    """
    Constructs the inverted index from a folder.
    Contains preprocesing for both documents,
    .csv files, and inputs.
    Contains inverted index testing methods.
    """

    inverted_index = {}
    # I use this throughout to alphanumerically rank data:
    convert = staticmethod(lambda text: int(text) if text.isdigit() else text.lower())

    def read_data(self, folder_path, doc_id_is_episode_name=True):
        """
        For an input folder with some .txt and .csv files, creates two dictionaries.
        In the first, the keys are the names of .txt files, and the value is the
        string inside the corresponding .txt file. In the second, the keys are
        the names of the .csv files, and the corresponding value is the set
        of elements of each row's second element of the .csv.

        path, str: the path to the folder with .txt and .csv files.
        doc_id_is_episode_name, bool: whether to make the keys of
        the text_dictionary the .txt name + the episode name,
        retrieved from the episode string.

        Returns:
        text_dictionary, dict: dictionary, where the keys are the names of
        .txt files, and the values are the string inside the txt.
        csv_features_dictionary, dict: dictionary, where the keys are the names of
        .csv files, and the value is the set of elements of each row's second
        element of the .csv.
        """

        def csv_to_set(path):
            """
            For an input path of a .csv file, creates a set,
            where elements are second entries in each row.
            A set is used to filter out the duplicate elements
            of the .csv. Variable names are explanatory.

            path, str: the path of the .csv file.

            Returns:
            elements, set: set of second entries in each
            row of .csv.
            """
            elements = []
            with open(path, "r") as file:
                reader = csv.reader(file)
                for row in reader:
                    elements.append(row[1])
            return set(elements)

        text_dictionary = (
            {}
        )  # where we store each .txt string with key as the file name
        csv_features_dictionary = (
            {}
        )  # where we store each .csv set with key as the file name
        print("Reading corpus in:")
        for file_name in tqdm(
            sorted(
                os.listdir(folder_path),
                key=lambda key: [
                    self.convert(c) for c in re.split("([0-9]+)", key)
                ],  # sorted first by season, then by episode
            )
        ):
            if file_name.endswith(".txt"):
                # Writing an explicit variable for epizode string to optionally make the name of the dict key
                # include the episode name - episode names contain stopwords which we may delete
                episode_string = open(os.path.join(folder_path, file_name)).read()
                # optionally naming the key e.g. '3.1 Stark Raving Dad'
                text_dictionary[
                    file_name.strip(".txt")
                    + " "
                    + episode_string[: episode_string.index("\n")]
                    if doc_id_is_episode_name
                    else file_name.strip(".txt")
                ] = open(os.path.join(folder_path, file_name), encoding="utf-8").read()
            elif file_name.endswith(".csv"):
                csv_features_dictionary[file_name] = csv_to_set(
                    os.path.join(folder_path, file_name)
                )
        return text_dictionary, csv_features_dictionary

    @staticmethod
    def preprocess_document(document, wikipedia=False, csv=False):
        """
        Preprocesses a string. For Wiki articles, removes
        code artefacts left, Wikipedia specific terms
        that only act as hyperlinks on the webpage,
        adds a specific expression instead of
        definite semantic breaks, replaces bold
        signs, and finally removes special
        characters, including numbers.
        For non wikis, expands contractions
        and removes special characters.
        Finally, the data is tokenised and
        stemmed with the Porter stemmer.
        Variable names are explanatory.

        document, str: string to preprocess
        wikipedia, bool: whether the string needs wikipedia
        special preprocessing
        csv, bool: whether the string needs csv
        special preprocessing

        Returns:
        tokens, list: tokens of document / inputs
        """

        def expand_contractions(text):
            """
            Expand all contractions in a given string.
            Variable names are explanatory.

            text, str: string to expand contractions in

            Returns:
            expanded_text, str: text with expanded contractions
            and removed apostrophess
            """

            def expand_match(contraction):
                """
                Expand a matched contraction.
                Variable names are explanatory.

                contraction, re.Match object: matched contraction in an str

                Returns:
                expanded_text, str: match with expanded contractions
                """
                # re.Match object, group(0) is the match str
                match = contraction.group(0)
                # in case it's lowercase
                first_char = match[0]
                # find the matching key and value from the map
                expanded_contraction = (
                    CONTRACTION_MAP.get(match)
                    if CONTRACTION_MAP.get(match)
                    else CONTRACTION_MAP.get(match.lower())
                )
                expanded_contraction = first_char + expanded_contraction[1:]
                return expanded_contraction

            expanded_text = CONTRACTIONS_PATTERN.sub(expand_match, text)
            expanded_text = re.sub("'", " ", expanded_text)
            return expanded_text

        def add_semantic_boundary_strings(document_string):
            """
            Replace semantic boundaries in a
            string with a separator string.

            document_string, str: string to process

            Returns:
            document_string, str: processed string
            """
            separator = " undeniablesemanticboundary "
            document_string = (
                re.sub("[?!]+", separator, document)
                .replace("...", separator)
                .replace("\n", separator)
            )
            return document_string

        if wikipedia:
            # encoding artifacts still appearing in some of the documents:
            document = re.sub(r"[^\x00-\x7F]+", " ", document)
            # replace wikipedia specific stuff that only works as hyperlinks and does not have semantic meaning:
            for a, b in WIKIPEDIA_MAP:
                document = document.replace(a, b)
            # add a special string ' semanticboundary ' for what
            # definitely separates the meaning of two words:
            # newlines, punctuation ? ! or ...
            # Can't be sure for simple periods.
            # Makes the sublist matching from .csv correct.
            document = add_semantic_boundary_strings(document)
            # \t is the bold symbol
            document = document.replace("\t", " ")
            document = expand_contractions(document)
            # remove all special characters (not sure why '[' and ']'
            # are not cut, so replace manually)
            document = (
                re.sub(r"[^a-zA-z\s]", " ", document)
                .replace("[", " ")
                .replace("]", " ")
            )
        else:  # if it's an input/csv entry
            if csv:
                document = expand_contractions(document)
                document = add_semantic_boundary_strings(document)
            document = expand_contractions(document)
            document = (
                re.sub(r"[^a-zA-z\s]", "", document).replace("[", " ").replace("]", " ")
            )
        tokens = word_tokenize(document)
        # to lowercase
        tokens = [w.lower() for w in tokens]  # must be list, not set
        # remove stopwords
        tokens = list(filter(lambda token: token not in ENGLISH_STOPWORDS, tokens))
        # finally stem with Porter stemmer, by each TOKEN
        # lemmatizer = WordNetLemmatizer() #chose not to use in the end
        stemmer = PorterStemmer()
        for index, term in enumerate(tokens):
            tokens[index] = stemmer.stem((term))
        return tokens

    # This splits and lowercases a search phrase if it's multi-word,
    # applies Norvig correction to each word,
    # applies preprocessing to each word,
    # and returns the rejoined string. I use this for both
    # dump and proximity search
    correct_and_preprocess_search_token = staticmethod(
        lambda raw_phrase, self: " ".join(
            self.preprocess_document(
                " ".join(
                    [
                        norvig_correction(uncorrected_term.lower())
                        for uncorrected_term in re.split(" ", raw_phrase)
                    ]
                )
            )
        )
    )

    def index_corpus(self, document_corpus, include_csv_features=True):
        """
        Constructs the inverted index by considering and
        preprocesing the documents and the csv entries.
        Adds multi-word entries to the index if they
        match the document's tokens which appear in
        succession.
        Variable names are explanatory.

        document_corpus, list: contains the
        dictionary, where the keys are the names of
        .txt files, and the values are the string
        inside the txts, as the first element, and
        the dictionary, where the keys are the names of
        and the value is the set of elements of each row's second
        element of the .csv as the second element.
        include_csv_features, bool: whether to include
        .csv features

        Returns:
        None
        """

        def add_features_to_token_dictionary(token_dictionary, features_set):
            """
            Inserts multi-word features from a feature set if they appear in
            a token dictionary, even if multiple times, right after the last token
            of the corresponding token word.
            Variable names are explanatory.

            token_dictionary, dict: dictionary with episodes as keys
            and preprocessed token lists as the values
            features_set, set: set of lists of preprocessed tokens
            from a specific .csv

            Returns:
            token_dictionary, dict: dictionary
            with updated documents
            """

            def find_sub_list_end_indices(sublist, larger_list):
                """
                Finds indices where a sublist terminates in a list.
                Variable names are explanatory.

                sublist, list: smaller list.
                larger_list, list: larger list.

                Returns:
                end_indices, list: indices of the larger list
                where sublist terminates.
                """
                end_indices = []
                sublist_length = len(sublist)
                for ind in (i for i, e in enumerate(larger_list) if e == sublist[0]):
                    if larger_list[ind : ind + sublist_length] == sublist:
                        end_indices.append((ind + sublist_length))
                return end_indices

            # token list for both characters and locations
            feature_token_list = []
            print("Preprocessing features from the .csv:")
            # Processing each entry in the csv
            for feature in tqdm(features_set):
                feature_token_list.append(self.preprocess_document(feature, csv=True))
            print(
                'Checking for occurance of the terms from the .csv and adding such multi-token terms as a single "token" to the documents:'
            )
            # go through the tokens of each episode
            for episode in tqdm((token_dictionary.keys())):
                episode_tokens = token_dictionary[
                    episode
                ]  # retrieve tokens of an episode
                for feature_tokens in feature_token_list:
                    if (
                        len(feature_tokens) > 1
                    ):  # non empty and more than one token (i.e. makes sense to add it as a separate multi-word entry!)
                        feature_token_indices_to_insert = find_sub_list_end_indices(
                            feature_tokens, episode_tokens
                        )  # i.e. we found where matches appear
                        new_token_to_insert = " ".join(
                            feature_tokens
                        )  # 'token made up of multiple words!
                        # the following makes sure that 'uncl moe famili feedbag'
                        # is added right after 'uncl', 'moe', 'famili', and 'feedbag'
                        # correctly even if such combination occurs more than once.
                        for (
                            index_of_index_to_insert_token,
                            index_to_insert_token,
                        ) in enumerate(feature_token_indices_to_insert):
                            token_dictionary[episode].insert(
                                feature_token_indices_to_insert[
                                    index_of_index_to_insert_token
                                ],
                                new_token_to_insert,
                            )
                            # increase the actual index where we insert the token
                            # feature_token_indices_to_insert[index_of_index_to_insert_token]
                            # by 1 in each iteration, as the size is increased by 1
                            # in each iteration.
                            feature_token_indices_to_insert = [
                                x + 1 for x in feature_token_indices_to_insert
                            ]
            return token_dictionary

        # retrieve
        document_dictionary, csv_features_dictionary = (
            document_corpus[0],
            document_corpus[1],
        )
        print("\nPreprocessing documents:")
        index_construction_start = time.time()
        # preprocess each string of an episode
        for episode in tqdm(document_dictionary):
            document_dictionary[episode] = self.preprocess_document(
                document_dictionary[episode], wikipedia=True
            )
        # here we add the character and the location *multi-term* tokens if they appear in the text

        if include_csv_features:
            for csv_features_dictionary_key in list(csv_features_dictionary.keys()):
                csv_set = csv_features_dictionary[csv_features_dictionary_key]
                print(
                    "\nComparing {0} to the document list.".format(
                        csv_features_dictionary_key
                    )
                )
                document_dictionary = add_features_to_token_dictionary(
                    document_dictionary, csv_set
                )
        token_sequence = []
        # for each episode string
        for episode in document_dictionary:
            # get the index of the term and the term
            semantic_boundary_counter = (
                0  # we will not include semantic boundaries as entries in the index...
            )
            for index, term in enumerate(document_dictionary[episode]):
                if term != "undeniablesemanticboundari":
                    # append the term, the episode, and the location number of that term/multi-term in an episode
                    # to a list of such entries for the whole corpus
                    term_episode_tuple = (
                        term,
                        episode,
                        index
                        - semantic_boundary_counter,  # ...so we substract the number of them we had previously in the document...
                    )
                    token_sequence.append(term_episode_tuple)
                else:
                    semantic_boundary_counter += 1  # ...and add one if it appears.
        print("\nSorting index...")
        index_sorting_start = time.time()
        # Sort first by term (alphabetically), then by docID (first by season, then by episode):
        token_sequence = sorted(
            token_sequence,
            key=lambda x: (x[0], [self.convert(c) for c in re.split("([0-9]+)", x[1])]),
        )
        print(
            "Index sorting took {0:4.2f} seconds.".format(
                (time.time() - index_sorting_start)
            )
        )
        inverted_index_dictionary = {}
        # Construct the inverted index
        for entry in token_sequence:
            term, doc_id, location = entry
            # if a term is new to the inverted index, add it as the key
            # to sub units Frequency (int) and 'Postings' (list of tuples
            # of doc_id and location), and initialize corresponding
            # values
            if term not in inverted_index_dictionary.keys():
                inverted_index_dictionary[term] = {}
                inverted_index_dictionary[term]["Frequency"] = 1
                inverted_index_dictionary[term]["Postings"] = [(doc_id, location)]
            # if a term is already in the inverted index, just append
            # the new pair of doc_id and corresponding location, and add 1 to freq.
            else:
                inverted_index_dictionary[term]["Frequency"] += 1
                inverted_index_dictionary[term]["Postings"].append((doc_id, location))
        print(
            "Final index has {0} terms and took {1:4.2f} seconds to construct.".format(
                len(inverted_index_dictionary.keys()),
                time.time() - index_construction_start,
            )
        )
        self.inverted_index = inverted_index_dictionary

    def dump(self, path_of_terms_to_check):
        """
        Spell-checks the validation terms,
        preprocesses them as a non-wiki entry,
        and retrieves the corresponding inverted
        index entries and how many episodes
        the term appears in.
        Variable names are explanatory.

        path_of_terms_to_check, str: path to .txt
        file where terms to look up are separated
        by a newline

        Returns:
        None
        """
        examples = open(path_of_terms_to_check).read().split("\n")
        print("Spell checking and looking up the validation terms:")
        for validation_entry in tqdm(examples):
            # correct and proces
            processed_and_corrected_entry_to_search = (
                self.correct_and_preprocess_search_token(validation_entry, self)
            )
            print(
                "\nEpisodes and corresponding locations where '{0}' occurs (searched as '{1}'):".format(
                    validation_entry, processed_and_corrected_entry_to_search
                )
            )
            try:
                # just look up the precessed term
                print(
                    self.inverted_index[processed_and_corrected_entry_to_search][
                        "Postings"
                    ]
                )
                # How many episodes does the term appear in?
                print(
                    "Total number of episodes that the term appears in is {0}.".format(
                        len(
                            set(
                                [
                                    x[0]
                                    for x in self.inverted_index[
                                        processed_and_corrected_entry_to_search
                                    ]["Postings"]
                                ]
                            )
                        )
                    )
                )
            except:
                print("No episodes found!")

    def proximity_search(self, term_1, term_2, window):
        """
        Searches two corrected and processed terms
        in the same document within a word number
        difference 'window'. Prints the ranked
        documents which are most likely to correspond
        to this query (have both terms within
        that window most times).
        Variable names are explanatory.

        term_1, str: first term to lookup
        term_2, str: second term to lookup

        Returns:
        None
        """
        # Get all of the episodes that a term appears in: won't use
        # convert_to_list_keys = lambda appearance_list: list(
        #     {a: b for a, b in appearance_list}.keys()
        # )
        # Correct and preprocess
        term_1_processed, term_2_processed = self.correct_and_preprocess_search_token(
            term_1, self
        ), self.correct_and_preprocess_search_token(term_2, self)
        print(
            "Documents within which '{0}' (searched as '{1}') and '{2}' (searched as '{3}') appear in specified window ranked:\n".format(
                term_1, term_1_processed, term_2, term_2_processed
            )
        )
        try:
            coexistence_dictionary = (
                {}
            )  # where we write the episode and how many times both terms appear in it within a window
            for term_1_episode, term_1_location in self.inverted_index[
                term_1_processed
            ][
                "Postings"
            ]:  # go thru were term 1 appears
                for term_2_episode, term_2_location in self.inverted_index[
                    term_2_processed
                ][
                    "Postings"
                ]:  # go thru were term 2 appears
                    if (term_1_episode == term_2_episode  # episode for both is the same
                        and abs(term_1_location - term_2_location) <= window  # the difference between locations smaller than window
                    ):
                        if (
                            term_1_episode not in coexistence_dictionary.keys()
                        ):  # term_1 or term_2, doesn't matter
                            coexistence_dictionary[
                                term_1_episode
                            ] = {}  # create empty dict
                            coexistence_dictionary[term_1_episode][
                                "Occurances"
                            ] = 1  # initialize occurances as 1
                            coexistence_dictionary[term_1_episode]["Positions"] = [
                                (term_1_location, term_2_location)
                            ]  # add the locations of both terms
                        else:
                            coexistence_dictionary[term_1_episode][
                                "Occurances"
                            ] += 1  # add 1 to the occurances
                            coexistence_dictionary[term_1_episode]["Positions"].append(
                                (
                                    term_1_location,
                                    term_2_location,
                                )  # append new location pair
                            )
            # Sort by occurances:
            coexistence_dictionary = sorted(
                coexistence_dictionary.items(), key=lambda x: -x[1]["Occurances"]
            )
            return coexistence_dictionary
        except:
            print("Either or both search terms do not appear within the corpus.")


##5. Construct inverted index and download as .csv for inspection or view as interactive pandas dataframe

In [ ]:
INPUT_FOLDER_PATH = "/content/simpsons_data" 
DOC_ID_IS_EPISODE_NAME = True  # Feel free to change to False. I know it would've been much smarter to create a map that links the .txt name to the episode name, but I don't have the time or the energy,
# and the difference between the .csv size is literally 2MB whether True or not
INCLUDE_CSV_FEATURES = True  # Can make to false to make faster in the development


def main():
    "main call function"
    index = InvertedIndex()  # initilaise the index
    corpus = index.read_data(
        folder_path=INPUT_FOLDER_PATH,
        doc_id_is_episode_name=DOC_ID_IS_EPISODE_NAME,
    )  # specify the directory path in which files are located
    index.index_corpus(
        corpus, include_csv_features=INCLUDE_CSV_FEATURES
    )  # index documents/corpus
    return index


index = main()


Reading corpus in:


  0%|          | 0/119 [00:00<?, ?it/s]


Preprocessing documents:


  0%|          | 0/117 [00:00<?, ?it/s]


Comparing simpsons_characters - row.csv to the document list.
Preprocessing features from the .csv:


...


Comparing simpsons_locations row.csv to the document list.
Preprocessing features from the .csv:


  0%|          | 0/4460 [00:00<?, ?it/s]

Checking for occurance of the terms from the .csv and adding such multi-token terms as a single "token" to the documents:


  0%|          | 0/117 [00:00<?, ?it/s]


Sorting index...
Index sorting took 0.83 seconds.
Final index has 10110 terms and took 92.56 seconds to construct.


Download the inverted index for inspection:



In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(index.inverted_index, orient = 'index')
df.to_csv('/content/test_inverted_index.csv')
from google.colab import files
files.download('test_inverted_index.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The output .csv file will have some artifacts after the lines where the number of episodes is extremely long (say after 'episod' or 'bart'), so you can inspect the df here (be sure to run the cell and press the magic wand!)

In [ ]:
df

,Frequency,Postings
aaron,1,"[(6.8 Lisa on Ice, 630)]"
abandon,17,"[(3.6 Like Father, Like Clown, 373), (3.6 Like..."
abat,1,"[(6.6 Treehouse of Horror V, 406)]"
abba,2,"[(7.8 Mother Simpson, 645), (7.8 Mother Simpso..."
abbey,1,"[(5.1 Homer's Barbershop Quartet, 1056)]"
...,...,...
zone,25,"[(3.7 Treehouse of Horror II, 120), (3.7 Treeh..."
zoo,5,"[(5.7 Bart's Inner Child, 423), (5.17 Bart Get..."
zoom,3,"[(7.9 Sideshow Bob's Last Gleaming, 747), (7.9..."
zorba,1,"[(3.14 Lisa the Greek, 639)]"


In [ ]:
# what long stuff have we added?
for term in list(index.inverted_index.keys()):
  if len(term.split(' ')) > 3:
    print(term)

georg h w bush
legitim businessman social club
lower east side springfield
shelbyvil nuclear power plant
springfield minimum secur prison
springfield nuclear power plant
springwood minimum secur prison
uncl moe famili feedbag


##6. Task 1 demos (dump)


Norvig correction demo:

In [ ]:
print(norvig_correction('homah'))
print(norvig_correction('humah'))
print(norvig_correction('voice-over'))

homer
human
voiceover


**To look up new terms when inspecting, add the term(s) of interest with a newline to the specified file.**

In [ ]:
validation_path = '/content/development-examples.txt'
index.dump(validation_path)

Spell checking and looking up the validation terms:


  0%|          | 0/19 [00:00<?, ?it/s]


Episodes and corresponding locations where 'Bart' occurs (searched as 'bart'):
[('3.1 Stark Raving Dad', 139), ('3.1 Stark Raving Dad', 287), ('3.1 Stark Raving Dad', 323), ('3.1 Stark Raving Dad', 375), ('3.1 Stark Raving Dad', 397), ('3.1 Stark Raving Dad', 402), ('3.1 Stark Raving Dad', 415), ('3.1 Stark Raving Dad', 429), ('3.1 Stark Raving Dad', 582), ('3.1 Stark Raving Dad', 592), ('3.1 Stark Raving Dad', 859), ('3.1 Stark Raving Dad', 862), ('3.1 Stark Raving Dad', 1084), ('3.1 Stark Raving Dad', 1108), ('3.1 Stark Raving Dad', 1213), ('3.1 Stark Raving Dad', 1739), ('3.1 Stark Raving Dad', 1771), ('3.2 Mr. Lisa Goes to Washington', 291), ('3.2 Mr. Lisa Goes to Washington', 556), ('3.2 Mr. Lisa Goes to Washington', 820), ('3.3 When Flanders Failed', 66), ('3.3 When Flanders Failed', 130), ('3.3 When Flanders Failed', 314), ('3.3 When Flanders Failed', 338), ('3.3 When Flanders Failed', 367), ('3.3 When Flanders Failed', 375), ('3.3 When Flanders Failed', 614), ('3.3 When Flande

##7. Task 2 demos (proximity search)

In [ ]:
TERM_1 = 'Homer Simpson'
TERM_2 = 'Bart'
WINDOW_SIZE = 100
#if you'd like to, but I guess the above is quicker:
# TERM_1 = input('Please enter term 1: ')
# TERM_2 = input('Please enter term 2: ')
# WINDOW_SIZE = input('Please enter the window size:')
index.proximity_search(term_1 = TERM_1, term_2 = TERM_2, window = WINDOW_SIZE)

Documents within which 'Homer Simpson' (searched as 'homer simpson') and 'Bart' (searched as 'bart') appear in specified window ranked:



[('5.5 Treehouse of Horror IV',
  {'Occurances': 12,
   'Positions': [(39, 50),
    (153, 156),
    (153, 238),
    (153, 248),
    (235, 156),
    (235, 238),
    (235, 248),
    (263, 238),
    (263, 248),
    (965, 894),
    (1531, 1503),
    (1531, 1535)]}),
 ('5.16 Homer Loves Flanders',
  {'Occurances': 2, 'Positions': [(838, 913), (843, 913)]}),
 ('6.17 Homer vs. Patty and Selma',
  {'Occurances': 2, 'Positions': [(577, 488), (577, 551)]}),
 ("5.1 Homer's Barbershop Quartet",
  {'Occurances': 1, 'Positions': [(128, 163)]}),
 ('5.9 The Last Temptation of Homer',
  {'Occurances': 1, 'Positions': [(1234, 1318)]}),
 ('7.1 Who Shot Mr. Burns?', {'Occurances': 1, 'Positions': [(558, 509)]})]

In [ ]:
TERM_1 = 'Homer Simpson'
TERM_2 = 'Bart Simpson'
WINDOW_SIZE = 100
#if you'd like to, but I guess the above is quicker:
# TERM_1 = input('Please enter term 1: ')
# TERM_2 = input('Please enter term 2: ')
# WINDOW_SIZE = input('Please enter the window size:')
index.proximity_search(term_1 = TERM_1, term_2 = TERM_2, window = WINDOW_SIZE)

Documents within which 'Homer Simpson' (searched as 'homer simpson') and 'Bart Simpson' (searched as 'bart simpson') appear in specified window ranked:



[('5.5 Treehouse of Horror IV',
  {'Occurances': 8,
   'Positions': [(39, 52),
    (153, 158),
    (153, 240),
    (235, 158),
    (235, 240),
    (263, 240),
    (1531, 1505),
    (1531, 1537)]}),
 ("5.1 Homer's Barbershop Quartet",
  {'Occurances': 1, 'Positions': [(128, 165)]})]

In [ ]:
TERM_1 = 'Tingting'
TERM_2 = 'Mu'
WINDOW_SIZE = 300
#if you'd like to, but I guess the above is quicker:
# TERM_1 = input('Please enter term 1: ')
# TERM_2 = input('Please enter term 2: ')
# WINDOW_SIZE = input('Please enter the window size:')
index.proximity_search(term_1 = TERM_1, term_2 = TERM_2, window = WINDOW_SIZE)

Documents within which 'Tingting' (searched as 'tingl') and 'Mu' (searched as 'mr') appear in specified window ranked:



[('5.14 Lisa vs. Malibu Stacy', {'Occurances': 1, 'Positions': [(858, 586)]})]

In [ ]:
TERM_1 = 'Goran'
TERM_2 = 'Nenadic'
WINDOW_SIZE = 100000
#if you'd like to, but I guess the above is quicker:
# TERM_1 = input('Please enter term 1: ')
# TERM_2 = input('Please enter term 2: ')
# WINDOW_SIZE = input('Please enter the window size:')
index.proximity_search(term_1 = TERM_1, term_2 = TERM_2, window = WINDOW_SIZE)

Documents within which 'Goran' (searched as 'moran') and 'Nenadic' (searched as 'nenad') appear in specified window ranked:

Either or both search terms do not appear within the corpus.


In [ ]:
TERM_1 = 'Patrick'
TERM_2 = 'Vanagas'
WINDOW_SIZE = 250
#if you'd like to, but I guess the above is quicker:
# TERM_1 = input('Please enter term 1: ')
# TERM_2 = input('Please enter term 2: ')
# WINDOW_SIZE = input('Please enter the window size:')
index.proximity_search(term_1 = TERM_1, term_2 = TERM_2, window = WINDOW_SIZE)

Documents within which 'Patrick' (searched as 'patrick') and 'Vanagas' (searched as 'manag') appear in specified window ranked:



[('5.12 Bart Gets Famous', {'Occurances': 1, 'Positions': [(872, 653)]}),
 ('5.17 Bart Gets an Elephant', {'Occurances': 1, 'Positions': [(976, 956)]})]

In [ ]:
TERM_1 = 'springfield'
TERM_2 = 'Aristotle'
WINDOW_SIZE = 100
#if you'd like to, but I guess the above is quicker:
# TERM_1 = input('Please enter term 1: ')
# TERM_2 = input('Please enter term 2: ')
# WINDOW_SIZE = input('Please enter the window size:')
index.proximity_search(term_1 = TERM_1, term_2 = TERM_2, window = WINDOW_SIZE)

Documents within which 'springfield' (searched as 'springfield') and 'Aristotle' (searched as 'aristotl') appear in specified window ranked:



[('3.5 Homer Defined',
  {'Occurances': 5,
   'Positions': [(102, 26), (224, 305), (257, 305), (734, 636), (734, 654)]}),
 ('6.23 The Springfield Connection',
  {'Occurances': 2, 'Positions': [(875, 866), (897, 866)]}),
 ('3.17 Homer at the Bat', {'Occurances': 1, 'Positions': [(252, 310)]})]

In [ ]:
TERM_1 = 'Gordie'
TERM_2 = 'Howe'
WINDOW_SIZE = 1
#if you'd like to, but I guess the above is quicker:
# TERM_1 = input('Please enter term 1: ')
# TERM_2 = input('Please enter term 2: ')
# WINDOW_SIZE = input('Please enter the window size:')
index.proximity_search(term_1 = TERM_1, term_2 = TERM_2, window = WINDOW_SIZE)

Documents within which 'Gordie' (searched as 'gordi') and 'Howe' (searched as 'howe') appear in specified window ranked:



[('3.16 Bart the Lover',
  {'Occurances': 6,
   'Positions': [(167, 168),
    (280, 281),
    (473, 474),
    (638, 639),
    (1101, 1102),
    (1115, 1116)]})]